# Additional features and functionalities of HOOMD-Organics

## Overview:
In this tutorial, we present additional features and functionalities within HOOMD-Organics, empowering users to customize their simulations according to their specific requirements. We will cover the following topics:

- Custom Molecule definition
- Custom ForceField definition
- Custom assembly algorithms
- Custom simulation protocols



In [ ]:
from hoomd_organics import Molecule

## Custom Molecule definition

You can define custom molecules in a couple of different ways:

1. Define molecules using their [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) string representation.
2. Utilize molecule files in formats such as `.mol` or `.sdf` to specify your custom molecules.
3. Create molecules from an [`mBuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`GMSO`](https://gmso.mosdef.org/en/stable/) topology.
4. Customize molecules by creating a subclass of the Molecule class



### Option 1: Using the SMILES string of the molecule

In [ ]:
benzoic_acid_mol = Molecule(num_mols=20, smiles="c1cc(C(O)=O)ccc1")

We use `mBuild` visualization function to visualize one of the 20 benzoic acid molecules.

In [ ]:
benzoic_acid_mol.molecules[0].visualize()

### Option 2: Initialize molecule from a file

In [ ]:
phenol_mol = Molecule(num_mols=20,
                      file="../hoomd_organics/assets/molecule_files/IPH.mol2")

In [ ]:
phenol_mol.molecules[0].visualize()

### Option 3: Start molecule from an [`mBuild`](https://mbuild.mosdef.org/en/stable/) compound or a [`GMSO`](https://gmso.mosdef.org/en/stable/) topology

In [ ]:
import mbuild as mb

mb_compound = mb.load("c1ccccc1", smiles=True)

gmso_top = mb_compound.to_gmso()

benzene_mol = Molecule(num_mols=20, compound=mb_compound)
benzene_mol = Molecule(num_mols=20, compound=gmso_top)


### Option 4: Define a subclass of the `Molecule` class

Checkout some polymer examples defined in `hoomd_organics/library/polymers.py`.

## Custom ForceFields

`HOOMD-Organics` package offers a range of predefined forcefield classes that ca be used to parameterize a system. Checkout the `hoomd_organics/library/forcefields.py` file to see the list of available forcefields.


To create a custom forcefield object, you can follow these two approaches:


- Defining a Force Field subclass: Alternatively, you have the flexibility to define your own custom forcefield class. Achieve this by creating a subclass of the `foyer.Forcefield` class or a class that generates a list of `hoomd.md.force` objects, tailored to your specific simulation requirements.

option 1: Using an XML File: If you possess an XML file of the desired forcefield, you can employ the `FF_from_file` class available in `hoomd_organics.library` to instantiate a forcefield object.

In [ ]:
from hoomd_organics.library import FF_from_file

benzene_ff = FF_from_file(
    xml_file="../hoomd_organics/assets/forcefields/benzene_opls.xml")

Checkout `hoomd_organics/library/forcefields.py` for more some examples of defining a forcefield using a subclass of `foyer.Forcefield` for specific molecules.

Option 2: Defining a Force Field subclass: Alternatively, you have the flexibility to define your own custom forcefield class. Achieve this by creating a subclass of the `foyer.Forcefield` class or a class that generates a list of `hoomd.md.force` objects, tailored to your specific simulation requirements.
Checkout `BeadSpring` class in  `hoomd_organics/library/forcefields.py` for an example of defining HOOMD force objects for a system of coarse-grained beads.

## Custom system assembly algorithms

Within the `HOOMD-Organics` package, the `System` class provides two distinct methods for filling the simulation box, Pack and Lattice. Here's a brief overview of each:

Pack Method:
The Pack method allows you to fill the simulation box with molecules in a way that closely packs them, typically in a disordered, random fashion.
Lattice Method:
The Lattice method, on the other hand, enables you to fill the simulation box using a well-defined lattice or grid structure.

Both `Pack` and `Lattice` classes are subclasses of the `System` class.

In [ ]:
# example of defining a system using the Lattice method
from hoomd_organics.base import Lattice
from hoomd_organics.library import PPS, OPLS_AA_PPS

pps = PPS(num_mols=32, lengths=5)

lattice = Lattice(molecules=pps, force_field=OPLS_AA_PPS(),
                  density=0.8,
                  r_cut=2.5,
                  x=1,
                  y=1,
                  n=4)

In [ ]:
lattice.system.visualize()

Note: base `System` class is designed as an abstract class, meaning it's not intended to be directly instantiated.
If you desire to customize molecule assembly algorithm that suits your specific requirements, you should create a subclass of the `System` class and override the abstract method `_build_system`. This method is responsible for organizing molecules from the `Molecule` class into a simulation box and returning the resulting mbuild compound. Checkout `Pack` and `Lattice` classes in `hoomd_organics/base/systems.py` for examples of how to define custom assembly algorithms.

##  Example of a system with multiple molecule types

`HOOMD-Organics` allows you to construct a system consisting of a mixture of molecule types, each potentially utilizing different force fields. If all the molecule types within the system share the same forcefield, then you only need to pass the forcefield once.

In [ ]:
from hoomd_organics.base import Pack
from hoomd_organics.library import OPLS_AA_DIMETHYLETHER
dimethylether_mol = Molecule(num_mols=20, smiles="COC")
pps_mol = PPS(num_mols=10, lengths=4)
multi_type_system = Pack(
    molecules=[dimethylether_mol, pps_mol],
    density=0.8,
    r_cut=2.5,
    force_field=[OPLS_AA_DIMETHYLETHER(), OPLS_AA_PPS()],
    auto_scale=True,
)

In [ ]:
multi_type_system.system.visualize()